In [3]:
import pandas as pd
# data = pd.read_csv('D:/ShipSupplies/DA/data/raw.csv')
# data = pd.read_csv('../data/동원산업통합_분류_1_RE.csv', encoding = 'cp949')
data = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')

In [4]:
# 결측치 확인
missing_values = data.isnull().sum()
print(missing_values)

청구서번호          0
No.            0
Subject        0
Machinery      0
Assembly       0
청구품목           0
Part No.1      0
Part No.2      0
key1           0
key2           0
리드타임_음수제거      0
청구량            0
견적             0
견적수량           0
견적화폐           0
견적단가           0
발주번호           0
발주처            0
발주             0
발주수량           0
발주금액           0
D/T            0
미입고 기간         0
리드타임           0
창고입고           0
창고입고수량         0
Control No.    0
입고창고           0
창고출고           0
창고출고수량         0
출고선박           0
출고운반선          0
선박입고           0
선박입고수량         0
완료 여부          0
dtype: int64


In [5]:
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   청구서번호        20517 non-null  object
 1   No.          20517 non-null  int64 
 2   Subject      20517 non-null  object
 3   Machinery    20517 non-null  object
 4   Assembly     20517 non-null  object
 5   청구품목         20517 non-null  object
 6   Part No.1    20517 non-null  object
 7   key1         20517 non-null  object
 8   key2         20517 non-null  object
 9   리드타임_음수제거    20517 non-null  int64 
 10  청구량          20517 non-null  int64 
 11  견적           20517 non-null  object
 12  견적수량         20517 non-null  int64 
 13  견적화폐         20517 non-null  object
 14  견적단가         20517 non-null  int64 
 15  발주번호         20517 non-null  object
 16  발주처          20517 non-null  object
 17  발주           20517 non-null  object
 18  발주수량         20517 non-null  int64 
 19  발주금액         20517 non-nu

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df = df.apply(le.fit_transform)

X = df[['Machinery', 'Assembly' , "Part No.1","청구품목"]]
y = df["key2"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True, random_state=42)

# XGBoost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

# 탐색할 하이퍼파라미터 그리드 축소
param_grid = {
    'max_depth': [3, 4],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200]
}

# GridSearchCV를 사용하여 하이퍼파라미터 탐색
grid_search = GridSearchCV(model, param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 확인
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
best_model = xgb.XGBClassifier(objective='multi:softmax', num_class=61, **best_params)
best_model.fit(X_train, y_train)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_class=61,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [23]:
# 예측 수행
y_pred = best_model.predict(X_test)

# 선택한 평가 지표 계산
f1 = f1_score(y_test, y_pred, average='macro')  # 예시로 F1-score 사용

# 결과 출력
print("F1-score:", f1)


F1-score: 0.812909827167555


In [ ]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = model.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

In [451]:
import pandas as pd
data = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [452]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
y = df['key2']
le = preprocessing.LabelEncoder()
y = y.values.ravel()  # y를 1차원 배열로 변환
y = le.fit_transform(y)

In [368]:
# # type(y),y.shape
# unique_values = np.unique(y)

# print(unique_values)
type(y)

numpy.ndarray

In [453]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['Machinery', 'Assembly' , "Part No.1","청구품목", 'key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [454]:
# df[['Machinery_encoded', 'Assembly_encoded' , "Part No.1_encoded","청구품목_encoded", 'key2_encoded' ]]

In [455]:
# 인코딩된 컬럼을 사용
encoded_columns = ['Machinery_encoded', 'Assembly_encoded', 'Part No.1_encoded', '청구품목_encoded']
X = df[encoded_columns].values
y = df['key2_encoded'].values

In [456]:
X,y

(array([[ 108,  281, 5759, 4848],
        [ 113, 1180, 5759, 3692],
        [ 113,  206, 5759, 5915],
        ...,
        [   0,  900, 8123, 4151],
        [   4, 1210, 8124, 1592],
        [   4, 1210, 8125, 4718]]),
 array([18, 18, 18, ...,  5, 20, 20]))

In [457]:
# 데이터를 학습용과 검증용으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [458]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print(X_train.shape)
print(X_train.dtype)


X_train shape: (16413, 4)
y_train shape: (16413,)
(16413, 4)
int32


In [459]:
# test 단어 집합의 크기
# max_test_idx_1 = int(np.max(X_test[:, 0]))
# max_test_idx_2 = int(np.max(X_test[:, 1]))
# max_test_idx_3 = int(np.max(X_test[:, 2]))
# max_test_idx_4 = int(np.max(X_test[:, 3]))

# print("Max test index for column 1:", max_test_idx_1)
# print("Max test index for column 2:", max_test_idx_2)
# print("Max test index for column 3:", max_test_idx_3)
# print("Max test index for column 4:", max_test_idx_4)

In [460]:
# train 단어 집합의 크기
# embedding_input_dim_1 = int(np.max(X_train[:, 0])) + 1
# embedding_input_dim_2 = int(np.max(X_train[:, 1])) + 1
# embedding_input_dim_3 = int(np.max(X_train[:, 2])) + 1
# embedding_input_dim_4 = int(np.max(X_train[:, 3])) + 1

# print(embedding_input_dim_1)
# print(embedding_input_dim_2)
# print(embedding_input_dim_3)
# print(embedding_input_dim_4)

In [461]:
import numpy as np

# 임베딩 설정
embedding_dim = 8

# 입력 정의
input1 = Input(shape=(1,), dtype="int32", name="Machinery_Input")
input2 = Input(shape=(1,), dtype="int32", name="Assembly_Input")
input3 = Input(shape=(1,), dtype="int32", name="Part_No.1_Input")
input4 = Input(shape=(1,), dtype="int32", name="청구품목_Input")

embedding_input_dim_1 = int(np.max(X_train[:, 0])) + 1
embedding_input_dim_2 = int(np.max(X_train[:, 1])) + 1
embedding_input_dim_3 = int(np.max(X_train[:, 2])) + 2 # Xtrain 길이가 더 길어야 하므로 읨의 조정
embedding_input_dim_4 = int(np.max(X_train[:, 3])) + 1


embedding1 = Embedding(embedding_input_dim_1, embedding_dim, name="Assembly_Embedding")(input1)
embedding2 = Embedding(embedding_input_dim_2, embedding_dim, name="Resource_Embedding")(input2)
embedding3 = Embedding(embedding_input_dim_3, embedding_dim, name="Method_Embedding")(input3)
embedding4 = Embedding(embedding_input_dim_4, embedding_dim, name="Dwg_Embedding")(input4)

## 모델 1 그냥 임베딩 -> 카테고리 예측

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, concatenate, Flatten

# 출력 통합
merged_output = concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1)

# Dense 계층
dense_output = Dense(1, activation='sigmoid', name='Dense_output')(merged_output)
output = Flatten()(dense_output)

# 예측 결과 계층
# prediction = Dense(1, activation="sigmoid", name="Prediction_output")(dense_output)

# y 를 원핫 인코딩 하면서 다중분류로 61개로 만드는 과정 
# prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

# 다중분류 출력 계층
# prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)


prediction = Dense(61, activation='softmax', name="Prediction_output")(output)
# 모델 생성
model = Model(inputs=[input1, input2, input3, input4], outputs=prediction)

## 모델 1 + CNN

In [377]:
from keras.layers import Input, Embedding, Dense, concatenate, Flatten, Conv1D, GlobalMaxPooling1D
from keras.models import Model,Sequential

merged_output = concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1)
conv_output = Conv1D(filters=128, kernel_size=3, activation='relu')(merged_output)
pool_output = GlobalMaxPooling1D()(conv_output)

dense_output = Dense(64, activation='relu', name='Dense_output')(pool_output)
prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

model = Model(inputs=[input1, input2, input3, input4], outputs=prediction)

# Sequential버전
# model = Sequential()
# model.add(concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1))
# model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(64, activation='relu', name='Dense_output'))
# model.add(Dense(61, activation='softmax', name="Prediction_output"))

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d. Consider increasing the input size. Received input shape [None, 1, 32] which would produce output shape with a zero or negative value in a dimension.

## 모델 1 + LSTM

In [405]:
from keras.layers import LSTM, Dense,Input, Embedding, concatenate, Flatten
from keras.models import Sequential, Model

merged_output = concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1)
lstm_output = LSTM(128)(merged_output)

dense_output = Dense(64, activation='relu', name='Dense_output')(lstm_output)
prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

model = Model(inputs=[input1, input2, input3, input4], outputs=prediction)

# # Sequential 버전
# model = Sequential()
# model.add(concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1))
# model.add(LSTM(128))
# model.add(Dense(64, activation='relu', name='Dense_output'))
# model.add(Dense(61, activation='softmax', name="Prediction_output"))



# model1 + cnn+ lstm

In [462]:



# 출력 통합
merged_output = concatenate([embedding1, embedding2, embedding3, embedding4], axis=-1)


# CNN Layer
# cnn_output = Conv1D(filters=128, kernel_size=1, activation='relu')(merged_output)  # kernel_size 수정
cnn_output = Conv1D(filters=128, kernel_size=7, activation='relu')(merged_output)  # kernel_size 수정
cnn_output = GlobalMaxPooling1D()(cnn_output)

# LSTM Layer
lstm_output = LSTM(128)(merged_output)

# Dense Layer
dense_input = concatenate([cnn_output, lstm_output], axis=-1)
dense_output = Dense(64, activation='relu', name='Dense_output')(dense_input)

prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

# 모델 생성
model = Model(inputs=[input1, input2, input3, input4], outputs=prediction)

In [463]:
from tensorflow.keras.optimizers import Adam
# 모델 컴파일
learning_rate = 0.001  # 원하는 학습률 값 설정하기
optimizer = Adam(learning_rate=learning_rate)
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"])
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Machinery_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 Assembly_Input (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Part_No.1_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 청구품목_Input (InputLayer)        [(None, 1)]          0           []                               
                                                                                           

In [464]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [466]:
import numpy as np
from tensorflow import keras

# 사용자 정의 콜백 클래스 정의
class EpochLogger(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 30 == 0:
            print(f"Epoch {(epoch + 1)}: Loss = {logs['loss']}, Accuracy = {logs['accuracy']}")

In [473]:
# 모델 훈련 및 콜백 설정
model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3]], y_train_categorical,
          epochs=50, batch_size=64, callbacks=[EpochLogger()])

Epoch 1/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0038 - accuracy: 0.9990
Epoch 2/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0039 - accuracy: 0.9990
Epoch 3/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0039 - accuracy: 0.9987
Epoch 4/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0040 - accuracy: 0.9987
Epoch 5/50
257/257 [==============================] - 1s 3ms/step - loss: 0.0041 - accuracy: 0.9985
Epoch 6/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0041 - accuracy: 0.9988
Epoch 7/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0039 - accuracy: 0.9987
Epoch 8/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0043 - accuracy: 0.9986
Epoch 9/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0038 - accuracy: 0.9984
Epoch 10/50
257/257 [==============================] - 1s 2ms/step - loss: 0.0030 - accuracy: 0.9990

In [474]:
y_pred = model.predict([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3]])

129/129 [==============================] - 0s 865us/step


In [475]:
y_pred_classes = np.argmax(y_pred, axis=1)
# 정답과 비교할 때 카테고리화 된 y_test 필요
y_test_classes = np.argmax(y_test_categorical, axis=1)

In [476]:
y_pred_classes,y_test_classes
# y_pred


(array([54, 33, 24, ..., 20, 25, 40], dtype=int64),
 array([54, 33, 24, ..., 51, 25, 40], dtype=int64))

In [477]:
# 정확도 계산
correct = np.sum(y_pred_classes == y_test_classes)
accuracy = correct / y_test_classes.shape[0]

print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 78.85%


In [478]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
print("F1 Score: {:.2f}%".format(f1 * 100))


F1 Score: 80.95%


In [361]:
# y_train_categorical
model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3]], y_train_categorical, epochs=30, batch_size=128)

Epoch 1/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4509 - accuracy: 0.5983
Epoch 2/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4508 - accuracy: 0.5986
Epoch 3/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4507 - accuracy: 0.5985
Epoch 4/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4506 - accuracy: 0.5984
Epoch 5/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4505 - accuracy: 0.5984
Epoch 6/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4504 - accuracy: 0.5985
Epoch 7/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4503 - accuracy: 0.5982
Epoch 8/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4503 - accuracy: 0.5984
Epoch 9/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4502 - accuracy: 0.5984
Epoch 10/30
129/129 [==============================] - 0s 1ms/step - loss: 1.4501 - accuracy: 0.5985

In [249]:
# 모델 학습
# model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3]], y_train, epochs=10, batch_size=32)
model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3]], y_train, epochs=30, batch_size=32)


Epoch 1/30
513/513 [==============================] - 1s 943us/step - loss: 1157.7864 - accuracy: 0.0016
Epoch 2/30
513/513 [==============================] - 1s 1ms/step - loss: 1151.3301 - accuracy: 0.0016
Epoch 3/30
513/513 [==============================] - 1s 1ms/step - loss: 1149.7228 - accuracy: 0.0016
Epoch 4/30
513/513 [==============================] - 1s 1ms/step - loss: 1148.9491 - accuracy: 0.0016
Epoch 5/30
513/513 [==============================] - 0s 922us/step - loss: 1148.5073 - accuracy: 0.0016
Epoch 6/30
513/513 [==============================] - 1s 982us/step - loss: 1148.2336 - accuracy: 0.0016
Epoch 7/30
513/513 [==============================] - 1s 976us/step - loss: 1148.0513 - accuracy: 0.0016
Epoch 8/30
513/513 [==============================] - 0s 937us/step - loss: 1147.9274 - accuracy: 0.0016
Epoch 9/30
513/513 [==============================] - 1s 1ms/step - loss: 1147.8386 - accuracy: 0.0016
Epoch 10/30
513/513 [==============================] - 0s 970us

In [265]:
# from tensorflow.keras.utils import to_categorical
# # y값 카테고리 화
# y_train_encoded = to_categorical(y_train, num_classes=61)
# y_test_encoded = to_categorical(y_test, num_classes=61)

In [266]:
import numpy as np

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

# X_train = np.expand_dims(X_train, axis=2)  # (데이터 개수, 시퀀스 길이, 1)
# X_test = np.expand_dims(X_test, axis=2)  # (데이터 개수, 시퀀스 길이, 1)

In [267]:
X_train.shape, X_test.shape

((16867, 30, 1, 1), (4217, 30, 1, 1))

In [268]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# 모델 구성
# model = Sequential()
# model.add(Embedding(input_dim=30, output_dim=100, input_length=30))
# model.add(Conv1D(32, kernel_size=3, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(61, activation='softmax'))
# model.add(Dense(4, activation='softmax'))


# model = Sequential()
# model.add(Conv1D(32, 3, activation='relu', input_shape=(30, 100)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(61, activation='softmax'))

model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=100, input_length=100))
model.add(Conv1D(32, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))


# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 100, 100)          300000    
                                                                 
 conv1d_20 (Conv1D)          (None, 98, 32)            9632      
                                                                 
 global_max_pooling1d_20 (Gl  (None, 32)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_35 (Dense)            (None, 128)               4224      
                                                                 
 dense_36 (Dense)            (None, 61)                7869      
                                                                 
Total params: 321,725
Trainable params: 321,725
Non-trainable params: 0
_______________________________________________

In [269]:
# 모델 학습
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
# model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded))
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
import matplotlib.pyplot as plt

# 손실 값 추출
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# 에포크 수
epochs = range(1, len(train_loss) + 1)

# 학습 곡선 그리기
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/10


ValueError: in user code:

    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of lay

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

NameError: name 'width' is not defined